In [1]:
from torch.utils.data import Dataset
from label_to_index import map_classes

class ProductDataset(Dataset):
    def __init__(self, products, img_transform=None):
        self.products = products
        self.img_transform = img_transform
        self.class_to_idx = map_classes()
        
    def __len__(self):
        return len(self.products)
    
    def __getitem__(self, idx):
        product = self.products[idx]
        img = product['image']
        if self.img_transform:
            img = self.img_transform(img)
        return img, self.class_to_idx[product['label']]

In [2]:
from torchvision import transforms
from random import shuffle
import pickle

augmented_data = pickle.load(open('augmented_data.p', 'rb'))
shuffle(augmented_data)

In [3]:
trfm = transforms.Compose([
                        transforms.Resize((64, 64)),
                        transforms.Grayscale(),
                        transforms.ToTensor(),
                        transforms.Normalize([0.48], [0.229])
        ])

train_dataset = augmented_data[:800]
test_dataset = augmented_data[800:1000]
val_dataset = augmented_data[1000:]

train_dataset = ProductDataset(train_dataset, trfm)
test_dataset = ProductDataset(test_dataset, trfm)
val_dataset = ProductDataset(val_dataset, trfm)


In [4]:
# Define the CNN Model 
# Source = https://github.com/meghanabhange/FashionMNIST-3-Layer-CNN/blob/master/Fashion.py
import torch
import torch.nn as nn

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        #Conv 1
        #Convolutio
        self.cnn1= nn.Conv2d(in_channels=1, out_channels= 16, kernel_size=5, stride=1, padding=2)
        #Activation
        self.relu1 = nn.ReLU()
        self.conv1_bn = nn.BatchNorm2d(16)
        #MaxPool1
        self.MaxPool1 = nn.MaxPool2d(kernel_size=2)
        
        #Conv 2
        self.cnn2 = nn.Conv2d(in_channels= 16, out_channels= 32, kernel_size=5,stride=1, padding=2)
        self.relu2 = nn.ReLU()
        self.conv2_bn = nn.BatchNorm2d(32)
        
        self.MaxPool2 = nn.MaxPool2d(kernel_size=2)
        
        self.cnn3 = nn.Conv2d(in_channels= 32, out_channels= 64, kernel_size=5,stride=1, padding=2)
        self.relu3 = nn.ReLU()
        self.conv3_bn = nn.BatchNorm2d(64)
        
        self.MaxPool3 = nn.MaxPool2d(kernel_size=2)
        
        self.fc1 = nn.Linear(4096, 11)
    
    
    def forward(self, x):
        
        out = self.cnn1(x)
        out= self.relu1(out)
        our = self.conv1_bn(out)
        
        out= self.MaxPool1(out)
        
        out= self.cnn2(out)
        out = self.relu2(out)
        out = self.conv2_bn(out)
        
        out = self.MaxPool2(out)
        
        out= self.cnn3(out)
        out = self.relu3(out)
        out = self.conv3_bn(out)
        
        out = self.MaxPool3(out)
        
        
        out = out.view(out.size(0), -1)
        out = self.fc1(out)
        
        return out

In [5]:
model = CNNModel()

In [6]:
from torch.utils.data import DataLoader

BATCH_SIZE = 20

train_img_loader = DataLoader(train_dataset, BATCH_SIZE)
test_loader = DataLoader(test_dataset, BATCH_SIZE)

In [7]:
import torch.optim as optim

optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()

In [8]:
def accuracy(preds, target):
    probs = torch.softmax(preds, dim=1)
    winners = probs.argmax(dim=1)
    corrects = (winners == target)
    accuracy = corrects.sum().float() / float(target.size(0))
    return accuracy

def train(model, img_loader, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0
    model.train()
    img_iter = iter(img_loader)
    for img, label in img_loader:
        optimizer.zero_grad()
        predictions = model(img).squeeze(1)
        loss = criterion(predictions, label)
        acc = accuracy(predictions, label)
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(img_loader), epoch_acc / len(img_loader)

def test_accuracy(model, test_loader):
    acc = 0
    for img, label in test_loader:
        predictions = model(img).squeeze(1)
        acc += accuracy(predictions, label)
        
        
    return acc.item() / len(test_loader)

In [9]:
for epoch in range(40):
    tl, ta = train(model, train_img_loader, optimizer, criterion)
    print("epoch - ", epoch)
    print("training loss and accuracy ", tl, ta)
    print("test accuracy ", test_accuracy(model, test_loader))

epoch -  0
training loss and accuracy  2.115928530693054 0.3087500019930303
test accuracy  0.31999998092651366
epoch -  1
training loss and accuracy  1.3960625186562539 0.5087499991059303
test accuracy  0.4349999904632568
epoch -  2
training loss and accuracy  1.0696466013789176 0.64375
test accuracy  0.5149999618530273
epoch -  3
training loss and accuracy  0.8860503420233726 0.6962499991059303
test accuracy  0.5199999809265137
epoch -  4
training loss and accuracy  0.762809981405735 0.7387500047683716
test accuracy  0.5549999713897705
epoch -  5
training loss and accuracy  0.6777146086096764 0.7650000035762787
test accuracy  0.6049999713897705
epoch -  6
training loss and accuracy  0.6233682829886675 0.783750006556511
test accuracy  0.6
epoch -  7
training loss and accuracy  0.5571248464286327 0.8062500044703483
test accuracy  0.5850000381469727
epoch -  8
training loss and accuracy  0.522036001458764 0.8137500062584877
test accuracy  0.6299999713897705
epoch -  9
training loss and a

In [10]:
val_loader = DataLoader(val_dataset, 1)
print("val accuracy", test_accuracy(model, val_loader))

val accuracy 0.66
